In [1]:
from astropy.time import Time
from PyAstronomy import pyasl
import numpy as np
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
%matplotlib tk
from datetime import datetime

The parameters of the functions below are:
* 'm1,m2': masses of primary and companion in units of solar mass.
* 'p': period in days.
* 'e': eccentricity.
* 'i': inclination in degrees.
* 'T0': time of periastron (in JD from Gaia- given for 2016.0 reference epoch)
* 'parallax': Gaia parallax in mas.

Tip for using the BinaryOrbit class:
* The z-axis is the line of sight from the system to the observer.
* The x-y plane is the projected plane we see.
* An orbit with 0 inclination is purely in the projected plane (z=const).
* All the units are MKS (meters, meters per second etc)

In [6]:
def get_days_since_periastron(t0,today):
    ref_epoch = Time('2016.0',format='jyear') ## gaia DR3 reference epoch in Julian years
    ref_epoch.format = 'jd'
    t1 = t0 + ref_epoch ## the absolute time of periastron in Julian days (t0 is relative to ref_epoch)
    t2 = Time(today, format='datetime')
    t2.format = 'jd'
    return t2 - t1

def plot_radial_velocity(m1, m2, p, e, i, T0, reference = datetime.today() ,plot_companion = False):
    bo = pyasl.BinaryOrbit(m2/m1, m2+m1, per = p , e = e, i = i) ## solve the orbit
    days = get_days_since_periastron(T0,reference)
    t_vec = np.arange(days.value,days.value+p,1)
    sec_per_day = 24*3600
    km_per_m = 1e-3
    v = bo.xyzVel(t_vec * sec_per_day)[0][:,2] ## get the z (radial) component of the primary velocity (meter/sec)
    v = v * km_per_m  ## convert to km/s

    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.plot(t_vec - days.value,v,label='primary',color='RoyalBlue')
    
    if plot_companion:
        v2 = bo.xyzVel(t_vec * sec_per_day)[1][:,2] ## get the z (radial) component of the companion velocity (meter/sec)
        v2 = v2 * km_per_m  ## convert to km/s
        ax.plot(t_vec - days.value,v2,label='companion',color='Crimson')
        ax.legend()
    ax.grid()
    ax.set_xlabel('t (days)',fontdict={'size':14})
    ax.set_ylabel(r'$v (km/s)$',fontdict={'size':14})
    ax.set_title('RV from today',fontdict={'size':14})
    fig.show()

def plot_projected_orbit(m1, m2, p, e, i, parallax, T0):
    bo = pyasl.BinaryOrbit(m2/m1, m2+m1, per = p , e = e, i = i) ## solve the orbit
    days = get_days_since_periastron(T0,datetime.today())
    t_vec = np.arange(days.value,days.value + p,1)
    sec_per_day = 24*3600
    distance = 1000 / parallax ## distance in parsec
    r1,r2 = bo.xyzPos(t_vec * sec_per_day) ## get the position of the primary and companion (meter) as a function of time
    x1 = r1[:,0] / (distance * u.pc).to_value('meter') * 206265 *1000 ## convert to milliarcsec
    y1 = r1[:,1] / (distance * u.pc).to_value('meter') * 206265 *1000 ## convert to milliarcsec
    x2 = r2[:,0] / (distance * u.pc).to_value('meter') * 206265 *1000 ## convert to milliarcsec
    y2 = r2[:,1] / (distance * u.pc).to_value('meter') * 206265 *1000 ## convert to milliarcsec

    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.plot(x1,y1,label='primary',color='RoyalBlue')
    ax.plot(x2,y2,label='companion',color='Crimson')
    ax.set_xlabel('x (mas)',fontdict={'size':14})
    ax.set_ylabel('y (mas)',fontdict={'size':14})
    ax.scatter(x1[0],y1[0],color='black',marker='X',s=50)
    ax.scatter(x2[0],y2[0],color='black',marker='X',s=50,label='today')
    ax.legend()
    ax.grid()
    fig.show()

def plot_projected_separation(m1, m2, p, e, i, parallax, T0):
    bo = pyasl.BinaryOrbit(m2/m1, m2+m1, per = p , e = e, i = i) ## solve the orbit
    days = get_days_since_periastron(T0,datetime.today())
    t_vec = np.arange(days.value,days.value + p,1)
    sec_per_day = 24*3600
    distance = 1000 / parallax ## distance in parsec
    r1,r2 = bo.xyzPos(t_vec * sec_per_day) ## get the position of the primary and companion (meter) as a function of time
    sep = np.linalg.norm((r1-r2)[:,:2],axis=1) ## the projected separation in the x-y plane (meter)
    sep = sep / (distance * u.pc).to_value('meter') * 206265 *1000 ## convert to milliarcsec
    
    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.plot(t_vec - days.value,sep)
    ax.set_xlabel('t (days)',fontdict={'size':14})
    ax.set_ylabel('separation (mas)',fontdict={'size':14})
    ax.set_title('projected separation from today',fontdict={'size':14})
    ax.grid()
    fig.show()


In [3]:
table_0 = Table.read('../table_B.fits',format='fits')

idx = 249
m1 = table_0['m1'][idx]
m1_err = table_0['m1_err'][idx]
m2 = table_0['m2'][idx]
m2_err = table_0['m2_err'][idx]
parallax = table_0['parallax'][idx]
distance = 1000/parallax

period = table_0['period'][idx]
period_err = table_0['period_error'][idx]

eccentricity = table_0['eccentricity'][idx]
eccentricity_err = table_0['eccentricity_error'][idx]

inclination = table_0['i_deg'][idx]
inclination_err = table_0['i_deg_error'][idx]

t0 = table_0['t_periastron'][idx]
t0_err = table_0['t_periastron_error'][idx]

In [5]:
print(f'm1 over error = {m1/m1_err}')
print(f'm2 over error = {m2/m2_err}')
print(f'period over error = {period/period_err}')
print(f'eccentricity over error = {eccentricity/eccentricity_err}')
print(f'inclination over error = {inclination/inclination_err}')
print(f't0 over error = {abs(t0/t0_err)}')

m1 over error = 11.068297915467632
m2 over error = 10.88638610960198
period over error = 63.898960344551284
eccentricity over error = 16.96621522770087
inclination over error = 230.06507501670214
t0 over error = 6.081876306165071


In [13]:
# lamost = datetime(year = 2017, month= 12, day = 22, hour =16, minute = 28, second = 3)
plot_radial_velocity(m1, m2, period , eccentricity, inclination, t0 , plot_companion = True)

In [99]:
plot_projected_orbit(m1, m2, period , eccentricity, inclination, parallax, t0)

In [10]:
plot_projected_separation(m1, m2, period , eccentricity, inclination, parallax, t0)